## engine + session

In [3]:
import pandas
import os
import sys
import re
import sqlalchemy
from flask import Flask, jsonify
from flask_restful import Api
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow

# init can also be done in one step as db = SQLAlchemy(app)
db = SQLAlchemy()
app = Flask(__name__)
# app configuration
app.config['FLASK_ENV'] = 'development'
app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get(
    'DATABASE_URL', 'postgresql://mahtin@localhost:5432/mahtin'
    )
# SQLalchemy has its own modification tracker which is better than the flask-sqlalchemy mod tracker
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['PROPAGATE_EXEPTIONS'] = True  # required to run errorhandlers
app.secret_key = 'mahtin'
data_path = os.environ.get(
    'DATA_PATH', '/Users/mahtin/Dropbox/Icke/Bilder/ClaraPix'
    )

api = Api(app)

db.init_app(app)

# can also be done in one step as ma = Marshmallow(app) in app.py
# ! needs to be done after db init
ma = Marshmallow(app)

/Users/mahtin/miniconda/envs/clara-env/lib/python3.7/site-packages/flask_marshmallow/__init__.py:27: UserWarning: Flask-SQLAlchemy integration requires marshmallow-sqlalchemy to be installed.
  "Flask-SQLAlchemy integration requires "


# Models

## ImageModel KeywordModel

In [4]:
# association table between image and keywords
image_keyword_table = db.Table('association', db.Model.metadata,
                               db.Column('img_id', db.Integer, db.ForeignKey('images.id', ondelete='cascade')),
                               db.Column('key_id', db.Integer, db.ForeignKey('keywords.id', ondelete='cascade'))
                               )

class ImageModel(db.Model):

    __tablename__ = 'images'

    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(90))
    path = db.Column(db.String(80), nullable=False, unique=True)
    date = db.Column(db.Date(), nullable=False)
    date_assumed = db.Column(db.Boolean(), nullable=True)
    note = db.Column(db.String(120))
    helper = db.Column(db.String(80))

    col_id = db.Column(db.Integer, db.ForeignKey('collections.id'))
    collection = db.relationship('CollectionModel', back_populates='images')

    user_id = db.Column(db.Integer, db.ForeignKey('users.id'))
    artist = db.relationship('UserModel', back_populates='images')

    keywords = db.relationship(
                               'KeywordModel',
                               secondary=image_keyword_table,
                               backref=db.backref('images', lazy='subquery'),
                               lazy=True,
                               # passive_deletes=True prevents deletions to be performed by SQLA
                               # but leaves that to the DB cascade --> delete is set on the DB as ondelete=CASCADE
                               passive_deletes=True
                               )

    # init method is not required anymore
    # def __init__(self, title, path, date, date_assumed, note, helper, stars):
    #     self.title = title
    #     self.path = path
    #     self.date = date
    #     self.date_assumed = date_assumed
    #     self.note = note
    #     self.helper = helper
    #     self.stars = stars

    @classmethod
    def find_by_id(cls, _id):
        return cls.query.filter_by(id=_id).first()

    @classmethod
    def find_by_title(cls, title):
        return cls.query.find_by(title=title).first()

    @classmethod
    def find_by_artist(cls, artist):
        return cls.query.find_by(artist=artist).all()

    @classmethod
    def find_by_keyword(cls, key_list):
        
        image_set = set()
        # use .options(selectinload('keywords'))
        for key in keyset:
            keyword = session.query(Keyword).filter_by(name=key).first()
            image_set.update(keyword.images)

    def save2db(self):
        db.session.add(self)
        db.session.commit()

    def delete_from_db(self):
        db.session.delete(self)
        db.session.commit(self)

# serialization is done by marshmallow
    # def json(self):
    #     return {
    #             'id': self.id,
    #             'title': self.title,
    #             'artis': self.artist.name,
    #             'date': self.date,
    #             'stars': self.stars,
    #             'keywords': [keyword.name for keyword in self.keywords],
    #             'collection': self.collection
    #     }

    def __repr__(self):
        return f'<Image("{self.title}" by {self.artist.name}, ~/{os.path.join(data_path, self.path)})>'
    
    
class Keyword(db.Model):
    __tablename__ = 'keywords'

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    images = db.relationship(
                               'KeywordModel',
                               secondary=image_keyword_table,
                               backref=db.backref('images', lazy='subquery'),
                               lazy=True  #,
                               # passive_deletes=True  # if delete is set on the DB as ondelete=CASCADE
                               )

    @classmethod
    def find_by_name(cls, keyword):
        return cls.query.filter_by(name=keyword).first()

    @classmethod
    def find_by_id(cls, _id):
        return cls.query.filter_by(id=_id).first()

    def save2db(self):
        db.session.add(self)
        db.session.commit(self)

    def delete_from_db(self):
        db.session.delete(self)
        db.session.commit()

    def json(self):
        images = [{} for image in self.images]

    def __repr__(self):
        return f"<Keyword(name={self.name}, images:{len(self.images)})>"

## Collection User

In [5]:
ImageModel.query

InvalidRequestError: When initializing mapper mapped class ImageModel->images, expression 'CollectionModel' failed to locate a name ("name 'CollectionModel' is not defined"). If this is a class name, consider adding this relationship() to the <class '__main__.ImageModel'> class after both dependent classes have been defined.

# DB Access

In [6]:
from sqlalchemy.orm import selectinload
keyword = db.session.query(Keyword).filter_by(name='Kunst').first()
for image in keyword.images:
    print(image.title, ' : ', image.keywords)
    


RuntimeError: No application found. Either work inside a view function or push an application context. See http://flask-sqlalchemy.pocoo.org/contexts/.

In [ ]:
keyset = ['Urzeit', 'Dinos']
image_set = set()
for key in keyset:
    keyword = session.query(Keyword).filter_by(name=key).first()
    image_set.update(keyword.images)
image_set

In [ ]:
session.query(Keyword.images).join(Image).filter_by(name = key).all()

In [7]:
session.query(Image).filter_by(title = 'Haus').first().keywords

NameError: name 'session' is not defined